# Generating Qubit Hamiltonians

In [7]:
from utility import *
import numpy as np 

In [8]:
qubit_transf = 'jw' # Jordan-Wigner transformations
lih = get_qubit_hamiltonian(mol='lih', geometry=1.5, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(lih))

H:
-4.103591882717407 [] +
-0.0036744565021407344 [X0 X1 Y2 Y3] +
-0.0028858915562681203 [X0 X1 Y2 Z3 Z4 Y5] +
0.0020342898427753073 [X0 X1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
-0.0028858915562681203 [X0 X1 X3 X4] +
0.0020342898427753073 [X0 X1 X3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
-0.005378238376570797 [X0 X1 Y4 Y5] +
0.0003167582147165302 [X0 X1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 Y11] +
0.0003167582147165302 [X0 X1 X5 Z6 Z7 Z8 Z9 X10] +
-0.0024548068286387725 [X0 X1 Y6 Y7] +
-0.0024548068286387712 [X0 X1 Y8 Y9] +
-0.0018103104313468535 [X0 X1 Y10 Y11] +
0.0036744565021407344 [X0 Y1 Y2 X3] +
0.0028858915562681203 [X0 Y1 Y2 Z3 Z4 X5] +
-0.0020342898427753073 [X0 Y1 Y2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
-0.0028858915562681203 [X0 Y1 Y3 X4] +
0.0020342898427753073 [X0 Y1 Y3 Z4 Z5 Z6 Z7 Z8 Z9 X10] +
0.005378238376570797 [X0 Y1 Y4 X5] +
-0.0003167582147165302 [X0 Y1 Y4 Z5 Z6 Z7 Z8 Z9 Z10 X11] +
0.0003167582147165302 [X0 Y1 Y5 Z6 Z7 Z8 Z9 X10] +
0.0024548068286387725 [X0 Y1 Y6 X7] +
0.0024548068286387712 [X0 Y1 Y8 X9]

This Qubit Hamiltonian encodes all of the $2^N$ eigenstates and eigenenergies, where $N=4$ is the number of qubits. 
In principle, one can represent the Hamiltonian as a matrix with $2^N$ dimensions and find the eigenvalues classically through diagonalization, but the cost of this approach clearly grows exponentially with $N$. 

Still, we can check that this approach indeed recovers the ground state energy against method in S1. 

In [9]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian H
n_qubits = openfermion.count_qubits(lih)
lih_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in lih.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    lih_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(lih_matrix)
print("The ground state energy from S1: ")
obtain_PES('lih', [1.5], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -7.882362286810939 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-7.88236229 -7.80553411 -7.80553411 ...  1.05835442  1.68680314
  1.94311812]


In [10]:
obtain_PES('lih', [1.5], 'sto-3g', 'fci')

E = -7.882362286810939 Eh


array([-7.88236229])

Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [11]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -3.947119127137703 [] +
0.029213980981022854 [X0] +
-0.003034656830196947 [X0 X1 Y2 Y3] +
0.008373361424254586 [X0 X1 Y2 Z3 Z6 Y7] +
0.0009148940971924874 [X0 X1 Z2 X3 Z4 Z5 Z7] +
-0.0014390450440280617 [X0 X1 Z2 Z4 Z5 Z6 X7] +
0.008373361424254586 [X0 X1 X3 X6] +
-0.0009148940971924874 [X0 X1 X3 Z6 Z7] +
0.005996760849732219 [X0 X1 X4] +
0.005996760849732217 [X0 X1 X5] +
-0.0307383271773184 [X0 X1 Y6 Y7] +
0.0014390450440280617 [X0 X1 X7] +
0.0009148940971924874 [X0 Y1 Y2] +
0.003034656830196947 [X0 Y1 Y2 X3] +
-0.0009148940971924874 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-0.008373361424254586 [X0 Y1 Y2 Z3 Z6 X7] +
0.0009148940971924874 [X0 Y1 Z2 Y3 Z4 Z5 Z7] +
-0.0014390450440280617 [X0 Y1 Z2 Z3 Y6] +
0.0014390450440280617 [X0 Y1 Z2 Z4 Z5 Y6 Z7] +
-0.0014390450440280617 [X0 Y1 Z2 Z4 Z5 Z6 Y7] +
0.008373361424254586 [X0 Y1 Y3 X6] +
-0.0009148940971924874 [X0 Y1 Y3 Z6 Z7] +
0.005996760849732219 [X0 Y1 Y4] +
0.005996760849732217 [X0 Y1 Y5] +
0.0307383271773184 [X0 Y1 Y6 X7

We can verify that this new Hamiltonian still encodes the ground state energy. 

In [12]:
# Building the matrix representation of the effective Hamiltonian
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]
